In [ ]:
import pymysql 

conn = pymysql.connect(host='localhost', user='root', password='0000', charset='utf8') 
cursor = conn.cursor() 

sql = "CREATE DATABASE IF NOT EXISTS library_management" 

cursor.execute(sql) 

conn.commit() 
conn.close() 

In [ ]:
import pymysql 

conn = pymysql.connect(host='localhost', user='root', password='0000', db='library_management', charset='utf8') 
cursor = conn.cursor() 

# ["도서번호","도서명", "저자", "출판사", "출판년도" ,"대출 상태"]

sql_book = """CREATE TABLE IF NOT EXISTS books (
book_id INT AUTO_INCREMENT PRIMARY KEY,
title VARCHAR(255) NOT NULL,
author VARCHAR(255) NOT NULL,
publisher VARCHAR(255) NOT NULL,
publication_year INT NOT NULL,
loaning BOOLEAN DEFAULT FALSE
)"""

sql_user = """CREATE TABLE IF NOT EXISTS users (
user_id INT AUTO_INCREMENT PRIMARY KEY,
username VARCHAR(50) NOT NULL,
phone_number VARCHAR(15) NOT NULL,
max_loan INT NOT NULL,
loaning INT NOT NULL DEFAULT 0
)"""

sql_loan = """CREATE TABLE IF NOT EXISTS loans (
loan_id INT AUTO_INCREMENT PRIMARY KEY,
user_id INT NOT NULL,
book_id INT NOT NULL,
loan_date DATE NOT NULL,
return_date DATE,
FOREIGN KEY (user_id) REFERENCES users(user_id) ON DELETE CASCADE,
FOREIGN KEY (book_id) REFERENCES books(book_id) ON DELETE CASCADE
)"""

cursor.execute(sql_book) 
cursor.execute(sql_user) 
cursor.execute(sql_loan) 

conn.commit() 
conn.close()   

In [ ]:
import pymysql 
from random import randint

conn = pymysql.connect(host='localhost', user='root', password='0000', db='library_management', charset='utf8') 
## 랜덤한 도서를 50개만드는 쿼리
try:
    with conn.cursor() as cursor:
        for i in range(1,51):
            # SQL 쿼리 실행하여 데이터 삽입
            sql = "INSERT INTO books (title, author, publisher, publication_year) VALUES (%s, %s, %s, %s)"
            book = (
                "도서 " + str(i),
                "저자 " + str(i),
                "출판사 " + str(randint(1,6)),
                str(2000 + randint(10,20))
            )
            cursor.execute(sql, book)
    # 변경사항 커밋
    conn.commit()

except Exception as e:
    print(f"데이터 삽입 중 오류 발생: {e}")
    conn.rollback()

In [ ]:
import pymysql 
from random import randint

user_labels = ["회원번호", "이용자명", "연락처", "대출가능 도서 수", "대출중인 도서 수"]

conn = pymysql.connect(host='localhost', user='root', password='0000', db='library_management', charset='utf8') 
## 랜덤한 이용자를 50개 만드는 쿼리
try:
    with conn.cursor() as cursor:
        for i in range(1,21):
            max_loan = randint(3,11)
            # SQL 쿼리 실행하여 데이터 삽입
            sql = "INSERT INTO users (username, phone_number , max_loan) VALUES (%s, %s, %s)"
            user = [
            "이용자 "+str(i),
            "010-"+str(randint(1000,9999))+"-"+str(randint(1000,9999)),
            str(max_loan)
            ]
            cursor.execute(sql, user)
    # 변경사항 커밋
    conn.commit()
except Exception as e:
    print(f"데이터 삽입 중 오류 발생: {e}")
    conn.rollback()

In [117]:
import pymysql 
import datetime
from random import randint

def check_and_loan_book(user_id, book_id):
    conn = pymysql.connect(host='localhost', user='root', password='0000', db='library_management', charset='utf8')
    cursor = conn.cursor()

    try:
        # 대출 중인 도서의 수 확인
        cursor.execute("SELECT COUNT(*) FROM loans WHERE user_id = %s", (user_id,))
        num_loans = cursor.fetchone()[0]

        # 사용자의 최대 대출 가능 도서 수 가져오기
        cursor.execute("SELECT max_loan FROM users WHERE user_id = %s", (user_id,))
        max_loan = cursor.fetchone()[0]

        # 대출이 가능한지 확인하고 대출 중인 도서 수를 증가시킴
        if num_loans < max_loan:
            # 이미 대출된 책인지 확인
            cursor.execute("SELECT loan_id FROM loans WHERE user_id = %s AND book_id = %s", (user_id, book_id))
            existing_loan = cursor.fetchone()

            if existing_loan:
                print("이미 대출된 책입니다.")
            else:
                # 대출 중인 도서 수 증가
                cursor.execute("UPDATE users SET loaning = %s WHERE user_id = %s", (num_loans + 1, user_id))

                # 도서의 대출 상태를 대출 중으로 변경
                cursor.execute("UPDATE books SET loaning = %s WHERE book_id = %s", (True, book_id))

                # 대출 기록 추가
                cursor.execute("INSERT INTO loans (user_id, book_id, loan_date) VALUES (%s, %s, CURDATE())", (user_id, book_id))

                print("도서 대출이 완료되었습니다.")
        else:
            print("대출이 불가능합니다. 대출 가능 도서 수를 초과하였습니다.")

        conn.commit()
    except Exception as e:
        print("에러 발생:", e)
        conn.rollback()
    finally:
        conn.close()


loan_labels = ["대출번호", "도서번호", "대출자번호", "대출일", "반납 예정일"]

conn = pymysql.connect(host='localhost', user='root', password='0000', db='library_management', charset='utf8') 
## 랜덤한 대출을 50개 만드는 쿼리

for i in range(1,51,2):
    start_date = datetime.datetime(2024, 1, 1)
    end_date = datetime.datetime(2024, 12, 31)
    # 랜덤한 날짜 생성
    random_date = start_date + datetime.timedelta(days=randint(0, (end_date - start_date).days))

    # 튜플 데이터 작성
    user_id = str(randint(1,20))
    book_id = str(i)
    loan_date = random_date.strftime('%Y-%m-%d')
    return_date = (random_date + datetime.timedelta(days=10)).strftime('%Y-%m-%d')
    
    # 도서, 유저 정보 변경
    check_and_loan_book(user_id, book_id)

대출이 불가능합니다. 대출 가능 도서 수를 초과하였습니다.
대출이 불가능합니다. 대출 가능 도서 수를 초과하였습니다.
대출이 불가능합니다. 대출 가능 도서 수를 초과하였습니다.
대출이 불가능합니다. 대출 가능 도서 수를 초과하였습니다.
대출이 불가능합니다. 대출 가능 도서 수를 초과하였습니다.
대출이 불가능합니다. 대출 가능 도서 수를 초과하였습니다.
대출이 불가능합니다. 대출 가능 도서 수를 초과하였습니다.
대출이 불가능합니다. 대출 가능 도서 수를 초과하였습니다.
대출이 불가능합니다. 대출 가능 도서 수를 초과하였습니다.
대출이 불가능합니다. 대출 가능 도서 수를 초과하였습니다.
대출이 불가능합니다. 대출 가능 도서 수를 초과하였습니다.
대출이 불가능합니다. 대출 가능 도서 수를 초과하였습니다.
대출이 불가능합니다. 대출 가능 도서 수를 초과하였습니다.
대출이 불가능합니다. 대출 가능 도서 수를 초과하였습니다.
대출이 불가능합니다. 대출 가능 도서 수를 초과하였습니다.
대출이 불가능합니다. 대출 가능 도서 수를 초과하였습니다.
대출이 불가능합니다. 대출 가능 도서 수를 초과하였습니다.
대출이 불가능합니다. 대출 가능 도서 수를 초과하였습니다.
대출이 불가능합니다. 대출 가능 도서 수를 초과하였습니다.
대출이 불가능합니다. 대출 가능 도서 수를 초과하였습니다.
대출이 불가능합니다. 대출 가능 도서 수를 초과하였습니다.
대출이 불가능합니다. 대출 가능 도서 수를 초과하였습니다.
대출이 불가능합니다. 대출 가능 도서 수를 초과하였습니다.
대출이 불가능합니다. 대출 가능 도서 수를 초과하였습니다.
대출이 불가능합니다. 대출 가능 도서 수를 초과하였습니다.
